In [1]:
from pathlib import Path
import numpy as np
import torch
import json
import joblib
from kaggle_environments import make
from kaggle_environments.envs.kore_fleets.kore_fleets import do_nothing_agent
from kaggle_environments.envs.kore_fleets.helpers import Board, Point, ShipyardAction, Shipyard, Fleet, Direction
import copy
import matplotlib.pyplot as plt
import seaborn as sns
import gym
from collections import deque

from env import Kore2022, get_stats_from_env
from processors import process_obs, process_action, deprocess_action, get_expected_kores_mined_per_step, get_attack_feature
from preprocess_data import process_one
from agent import Agent
from configs import configs
from utils import *
from net import Categorical

In [2]:
for path in Path('data/raw').glob('*.json'):
    path = str(path)
    if 'info' not in path:
        with open(path, 'r') as f:
            match = json.load(f)
        if len(match['steps']) > 300:
            break

/home/kimyoonsoo/anaconda3/envs/kore2022/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [3]:
step = 310
player = np.argmax(match['rewards'])
conf = match['configuration']
obs = match['steps'][step][0]['observation']
obs['player'] = player
action = match['steps'][step+1][player]['action']

In [4]:
board = Board(obs, conf)

In [ ]:
board = board.next()

In [25]:
obs['players'][player]

[6382.8319325396815,
 {'0-2': [330, 0, 310],
  '111-3': [324, 0, 199],
  '140-2': [285, 0, 170],
  '147-3': [269, 0, 163],
  '177-5': [388, 0, 133],
  '223-4': [203, 0, 87],
  '230-3': [96, 62, 81],
  '264-2': [255, 0, 47],
  '271-4': [66, 0, 40],
  '278-6': [147, 0, 33],
  '281-3': [423, 0, 30],
  '282-7': [42, 0, 29],
  '285-5': [156, 16, 25],
  '290-2': [123, 0, 20],
  '292-5': [30, 0, 18],
  '308-6': [215, 36, 3]},
 {'299-1': [91, 114.741, 8, 3, 'E'],
  '299-3': [99, 98.85599999999998, 21, 3, ''],
  '299-4': [138, 125.358, 22, 0, ''],
  '300-2': [338, 151.98499999999999, 21, 3, '7NW'],
  '300-5': [87, 149.384, 21, 0, ''],
  '301-1': [339, 86.494, 9, 1, 'N'],
  '301-2': [252, 42.28600000000001, 8, 1, ''],
  '302-1': [276, 88.177, 14, 0, ''],
  '303-2': [101, 70.391, 21, 0, '2ES'],
  '306-2': [177, 144.641, 13, 1, 'N'],
  '306-3': [60, 76.371, 10, 1, 'S'],
  '306-4': [29, 71.98599999999999, 18, 2, 'E'],
  '307-1': [94, 19.263, 13, 2, '3W'],
  '307-3': [176, 95.894, 21, 1, '1N'],
  '3

In [26]:
%%time
policy_input, value_input, mask = process_obs(obs, conf)
raw_action = deprocess_action(action, obs, conf)

CPU times: user 736 ms, sys: 1.32 ms, total: 737 ms
Wall time: 741 ms


In [27]:
print(np.where(policy_input[0, 16] != policy_input[0, 3]))
print(np.where(policy_input[0, 17] != policy_input[0, 10]))

(array([10]), array([5]))
(array([], dtype=int64), array([], dtype=int64))


In [28]:
process_action(raw_action, obs, conf)

{'0-2': '',
 '111-3': '',
 '140-2': '',
 '147-3': '',
 '177-5': '',
 '223-4': '',
 '230-3': 'LAUNCH_21_N1E3S1W',
 '264-2': '',
 '271-4': '',
 '278-6': '',
 '281-3': '',
 '282-7': '',
 '285-5': 'LAUNCH_16_W1SE1N',
 '290-2': '',
 '292-5': '',
 '308-6': 'SPAWN_2'}